# X-ray
* 저희는 `torchxrayvision` 모델을 사용했습니다.
* 코드를 실행시키기 위해, `torchxrayvision` repository를 clone 해와야 합니다.

In [3]:
! git clone https://github.com/mlmed/torchxrayvision.git

Cloning into 'torchxrayvision'...
remote: Enumerating objects: 1470, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1470 (delta 27), reused 35 (delta 14), pack-reused 1406
Receiving objects: 100% (1470/1470), 42.68 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (861/861), done.


* 환경설정

In [ ]:
! cd torchxrayvision && \
pip install -r requirements.txt && \
python setup.py install

* torchxrayvision을 사용하기 위해, 본 ipynb 파일은 `torchxrayvision/scripts` 폴더 내에 위치해야 합니다. 파일 위치를 해당 위치로 이동시켜야 합니다.
* 본 `Xray.ipynb`파일을 `torchxrayvision/scripts/Xray.ipynb`에 위치하게 해주세요.
* 파일을 이동한 후, 아래 코드블록부터 시작해주세요.

In [5]:
import torch
import torchxrayvision as xrv
import skimage, torch, torchvision
import numpy as np

model = xrv.models.DenseNet(weights="densenet121-res224-all")

In [3]:
NORMAL = 0
ABNORMAL = 1
DISEASES = ['Effusion','Pneumonia','Pneumothorax']

def inference(img_path, THRESHOLD):
    global model
    img = skimage.io.imread(img_path)
    img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
    if len(img.shape) == 2:
        img = np.stack((img,)*3, axis=-1)
    img = img.mean(2)[None, ...] # Make single color channel
    
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    
    img = transform(img)
    img = torch.from_numpy(img)
    img = img.cuda()
    
    outputs = model(img[None,...]) # or model.features(img[None,...]) 
    outputs = dict(zip(model.pathologies,outputs[0].cpu().detach().numpy()))
    
    predict = NORMAL
    for disease in DISEASES:
        if outputs[disease] > THRESHOLD:
            predict = ABNORMAL
    return predict
    

* 모델 및 inference 코드

In [ ]:
model = xrv.models.DenseNet(weights="densenet121-res224-all")

def inference(img_path, THRESHOLD):
    global model
    img = skimage.io.imread(img_path)
    img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
    if len(img.shape) == 2:
        img = np.stack((img,)*3, axis=-1)
    img = img.mean(2)[None, ...] # Make single color channel
    
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    
    img = transform(img)
    img = torch.from_numpy(img)
    img = img.cuda()
    
    outputs = model(img[None,...]) # or model.features(img[None,...]) 
    outputs = dict(zip(model.pathologies,outputs[0].cpu().detach().numpy()))
    
    predict = NORMAL
    for disease in DISEASES:
        if outputs[disease] > THRESHOLD:
            predict = ABNORMAL
    return predict
    


In [ ]:
submission = pd.read_csv('/path/to/1000_sample_submission.csv')
submission

In [ ]:
model.cuda()
model.eval()


test_list = sorted(os.listdir('/path/to/XrayTestFolder'))

with torch.inference_mode():
    for img_path in tqdm.tqdm(test_list):
        predicted = inference(os.path.join('/home/hackathon/jupyter/storage/datasets/ChestPA/ChestPA_Test/test', img_path),THRESHOLD=0.51)
        submission.loc[submission['filename']==img_path,'result'] = predicted

submission.to_csv('./result.csv',index=False)